In [3]:
from pathlib import Path
from utilities import files
import os.path as op
import numpy as np
from os import sep, remove
import itertools as it
import json
from mne import read_epochs
import matlab.engine
import shutil

In [17]:
def many_is_in(multiple, target):
    check_ = []
    for i in multiple:
        check_.append(i in target)
    return any(check_)

def all_is_in(multiple, target):
    check_ = []
    for i in multiple:
        check_.append(i in target)
    return all(check_)

def cat(options, target):
    for i in options:
        if i in target:
            return i

def get_res4(ds_path, sub, ses, run):
    ref_path = op.join(ds_path, "raw", sub, ses, "meg")
    blocks = files.get_folders_files(ref_path)[0]
    block = [i for i in blocks if "block-{}".format(run[1:]) in i][0]
    res4_path = files.get_files(block, "", ".res4")[2][0]
    return res4_path

In [38]:
dataset_path = "/home/common/bonaiuto/multiburst/derivatives/processed"
subjects_info = "/home/common/bonaiuto/multiburst/participants.tsv"
subject_paths = files.get_folders_files(dataset_path)[0]
singularity_path = "/scratch/freesurfer_lic.sif"
og_dataset = "/home/common/bonaiuto/cued_action_meg/"

In [6]:
n_layers = 11
layers_name = "multilayer_{}".format(str(n_layers).zfill(2))
subject_path = subject_paths[0]
subject = subject_path.split(sep)[-1]
print(subject_path, subject)
output_folder = op.join(subject_path, layers_name)
files.make_folder(output_folder)

/home/common/bonaiuto/multiburst/derivatives/processed/sub-001 sub-001


In [7]:
exclude=[
    "sub-002-ses-01",
    "sub-002-ses-02",
    "sub-006-ses-01",
    "sub-006-ses-02",
    "sub-008-ses-01"
]

In [8]:
fs_ds_path = "/home/common/bonaiuto/cued_action_meg/derivatives/processed/"
fs_path = op.join(fs_ds_path, subject, "fs")

In [20]:
parasite = matlab.engine.start_matlab()

In [ ]:
# create multilayer
parasite.create_multilayer_surface(
    fs_path, n_layers, output_folder, 
    layers_name, singularity_path, "/home/common/bonaiuto/",
    nargout=0
)


In [ ]:
# prepare surfaces
parasite.prepare_surfaces(
    fs_path, output_folder,
    nargout=0
)

In [49]:
# prepare multilayer
# works in MATLAB
parasite.prepare_multilayer_surface(
    fs_path, output_folder, float(n_layers), layers_name,
    nargout=0
)

In [ ]:
# intermediate steps moving
intm_path = op.join(output_folder, "inter")
files.make_folder(intm_path)
files_paths = files.get_files(output_folder, "", "")[2]
lyrs = np.arange(n_layers)[1:-1]
lyrs = [str(i)for i in lyrs]
files_input = [i for i in files_paths if many_is_in(lyrs, i.split(sep)[-1])]
files_input = [i for i in files_input if layers_name not in i]
files_output = [op.join(intm_path, i.split(sep)[-1]) for i in files_paths if many_is_in(lyrs, i.split(sep)[-1])]
files_io = list(zip(files_input, files_output))
[shutil.move(*i) for i in files_io];

In [18]:
session_path = op.join(subject_path, "ses-01")
session_files = files.get_files(session_path, "autoreject", ".fif")[2]

In [33]:
def average_filter_convert(file_path, ds_path, parasite, filt=False, l_freq=None, h_freq=None):
    path_split = file_path.split(sep)
    filename_core = path_split[-1].split(".")[0]
    sub = filename_core[11:18]
    ses = filename_core[19:25]
    run = filename_core[26:29]
    res4_path = get_res4(ds_path, sub, ses, run)
    dir_path = str(sep).join(path_split[:-1] + ["avg_spm", ""])
    files.make_folder(dir_path)
    
    filt_status = "_no_filter"
    if filt:
        filt_status = "_filt"
    
    output_file = "spm-converted{}_{}".format(filt_status, filename_core)
    output_path = op.join(dir_path, output_file)
    average_file = output_path + "-ave.fif"
    mat_output = output_path + ".mat"
    if not op.exists(mat_output):
        if not op.exists(average_file):
            epochs = read_epochs(file_path, verbose=False)
            epochs = epochs.average()
            if filt:
                epochs.filter(l_freq=l_freq, h_freq=h_freq)
            epochs.save(average_file)

        parasite.convert_mne_to_spm(res4_path, average_file, mat_output, 0, nargout=0)
        if op.isfile(average_file):
            remove(average_file)
        else:
            print(average_file, "does not exists")
        print(filename_core, "converted")

    else:
        print(mat_output, "exists")
        
    return mat_output

In [34]:
mat_files = [average_filter_convert(i, og_dataset, parasite) for i in session_files];

/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/ses-01/avg_spm/spm-converted_no_filter_autoreject-sub-001-ses-01-001-motor-epo.mat exists
/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/ses-01/avg_spm/spm-converted_no_filter_autoreject-sub-001-ses-01-001-visual-epo.mat exists
/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/ses-01/avg_spm/spm-converted_no_filter_autoreject-sub-001-ses-01-002-motor-epo.mat exists
/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/ses-01/avg_spm/spm-converted_no_filter_autoreject-sub-001-ses-01-002-visual-epo.mat exists
/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/ses-01/avg_spm/spm-converted_no_filter_autoreject-sub-001-ses-01-003-motor-epo.mat exists
/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/ses-01/avg_spm/spm-converted_no_filter_autoreject-sub-001-ses-01-003-visual-epo.mat exists


In [36]:
inverted_output = op.join(output_folder, "inverted")
subjects_info

mu_file = op.join(inverted_output, )
it_file = op.join(inverted_output, )
res_file = op.join(inverted_output, )

In [ ]:
def invert_multisurface(inverted_output, t1_file, mat_file, subjects_info, parasite, layers=n_layers):
    files.make_folder(inverted_output)
    input_path = Path(mat_file)
    bits = input_path.name.split("_")[-1].split("-")
    sub = "-".join([bits[1], bits[2]])
    epo = bits[6]
    ses = "-".join([bits[3], bits[4]])
    run = bits[5]
    link_vector = files.get_files(
        op.join(Path(inverted_output).parent), 
        "multilayer", ".ds.link_vector.nodeep.gii"
    )[2][0]
    mu_file = op.join(inverted_output, "MU_" + input_path.stem + ".tsv")
    it_file = op.join(inverted_output, "IT_" + input_path.stem + ".tsv")
    res_file = op.join(inverted_output, "res_" + input_path.stem + ".tsv")
    json_out_file = op.join(inverted_output, "invert-res_" + input_path.stem + ".json")
#     print(inverted_output, subjects_info, mat_file, t1_file, link_vector, mu_file, it_file, res_file, json_out_file, layers, sub, ses, run, epo)
    parasite.invert_multisurface(
        inverted_output, subjects_info, mat_file, t1_file, 
        link_vector, mu_file, it_file, res_file, 
        json_out_file, float(layers), sub, ses, run, epo
    )
    
t1_file = "{}/raw/{}/mri/headcast/t1w.nii".format(og_dataset, subject)
invert_multisurface(inverted_output, t1_file, mat_files[0], subjects_info, parasite)

True

In [61]:
ml_vector

'/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/multilayer_11.ds.link_vector.nodeep.gii'